In [2]:
import numpy as np

In [4]:
import cPickle as cp
# with open('../../fast-rcnn/output/car_ds/cars_val/vgg16_fast_rcnn_iter_1800/test_results_hc44c.pkl', 'rb') as f:
#     dt = [np.array(l) for l in cp.load(f)[1]]
with open('../../fast-rcnn/data/OIRDS_v1_0/OIRDS_val.pkl', 'rb') as f:
    raw_gt = cp.load(f)
pics = sorted(raw_gt.keys())
gt = [np.array(raw_gt[k]) for k in pics]

In [29]:
with open('../../fast-rcnn/output/car_ds/cars_test/vgg16_fast_rcnn_iter_2000/nms_detections.pkl', 'rb') as f:
    dt = [np.array(l) for l in cp.load(f)[1]]

In [19]:
with open('../../fast-rcnn/data/OIRDS_v1_0/OIRDS_train.pkl', 'rb') as f:
    raw_gt = cp.load(f)
pics = sorted(raw_gt.keys())
train_gt = [np.array(raw_gt[k]) for k in pics]
with open('../../fast-rcnn/data/OIRDS_v1_0/OIRDS_test.pkl', 'rb') as f:
    raw_gt = cp.load(f)
pics = sorted(raw_gt.keys())
test_gt = [np.array(raw_gt[k]) for k in pics]

In [40]:
# gets IoUs for a single pic
def get_ious(gt, dt):
    """
    output 
        -a DxG grid of IoUs (D = num of detections, G = num of ground truths)
        -score vector for said detections
    """ 
    ious = np.zeros((len(dt),len(gt)), dtype=np.float32)
    x1_dt, x1_gt = dt[:, 0], gt[:, 0]
    y1_dt, y1_gt = dt[:, 1], gt[:, 1]
    x2_dt, x2_gt = dt[:, 2], gt[:, 2]
    y2_dt, y2_gt = dt[:, 3], gt[:, 3]
    scores       = dt[:, 4][:, np.newaxis]
    
    areas_dt = (x2_dt - x1_dt + 1) * (y2_dt - y1_dt + 1)
    areas_gt = (x2_gt - x1_gt + 1) * (y2_gt - y1_gt + 1)
    
    for i in xrange(len(dt)):
        xx1 = np.maximum(x1_dt[i], x1_gt)
        yy1 = np.maximum(y1_dt[i], y1_gt)
        xx2 = np.minimum(x2_dt[i], x2_gt)
        yy2 = np.minimum(y2_dt[i], y2_gt)
        
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        
        ovr = inter / (areas_dt[i] + areas_gt - inter)
        ious[i] = ovr
    return np.hstack([ious, scores])

In [41]:
def get_accuracy(ious, iou_thres=0.5, score_thres=0.5):
    raw_z = []
    raw_s = []
    for i in xrange(len(ious)):
        D, G = ious[i].shape
        # sanity check
        for j in xrange(D):
            assert((ious[i][j, :-1] >= iou_thres).sum() <= 1)
        for j in xrange(G - 1):
            assert((ious[i][:, j] >= iou_thres).sum() <= 1)

        # build z vector
        zi = np.zeros(D)
        # fill zi with ones at locations where a match was found
        for j in xrange(D):
            idx = np.argmax(ious[i][j, :-1])    
            if ious[i][j, idx] >= iou_thres:
                zi[j] = 1
        raw_z.append(zi)

        # build s vector
        # fill si with ones at locations above score threshold
        si = ious[i][:, -1] >= score_thres
        raw_s.append(si)

    z = np.hstack(raw_z)
    s = np.hstack(raw_s)
    # N = sum([len(gt[j]) for j in xrange(len(gt))])

    accuracy = np.sum(np.dot(s,z)) / np.sum(s)
    # recall = sum(np.dot(s,z)) / N
    return accuracy #, recall

In [42]:
def get_recall_thresholds(gt, dt):
    raw_scores = np.unique(np.hstack([dt[j][:, 4] for j in xrange(len(dt))]))
    scores = np.sort(raw_scores)
    thresholds = np.zeros(len(scores)-1)
    for j in xrange(len(thresholds)):
        cur_thresh = (scores[j]+scores[j+1])/2
        thresholds[j] = cur_thresh
    return thresholds[::2]

In [49]:
import time
t0 = time.time()
iou_thres = 0.5
# print(dt)
recall_thres = get_recall_thresholds(gt, dt) # ; print(len(recall_thres))
ious = [get_ious(gt[j], dt[j]) for j in xrange(len(gt))]
accuracy = [get_accuracy(ious, iou_thres=iou_thres, score_thres=t) for t in recall_thres]
print('mAP={} at IoU threshold {}, in time {:.02f}'.format(np.mean(accuracy), iou_thres, time.time()-t0))

mAP=0.53851446959 at IoU threshold 0.5, in time 1.41


In [29]:
arr = np.arange(30)
arr[::2]

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28])

In [103]:
# obsolete code
num_pics = len(gt)
thresholds = np.arange(0.5,0.95,0.05)
num_thresh = thresholds.shape[0]
accuracy = np.zeros((num_thresh,num_pics))
for i in xrange(num_thresh):
    for j in xrange(num_pics):
        ious = get_ious(gt[j],dt[j])
        thresh = 0.5
        count = 0.0
        for k in xrange(ious.shape[0]):
            _gt = np.argmax(ious[k])
            iou = ious[k,_gt]
            if iou >= thresholds[i]:
                count += 1
        accuracy[i,j] = count / ious.shape[0]
mean_accuracy = np.mean(accuracy)

In [108]:
ious = get_ious(gt[j],dt[j]); ious

array([[ 0.86496615,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]], dtype=float32)

In [106]:
for i in xrange(len(accuracy)): print(accuracy[i])

[ 1.          1.          0.16666667  0.75        0.          0.14285714
  0.4         0.05882353  0.25        0.2         0.125       0.25        1.
  1.          1.          1.          0.16666667  0.16666667  0.33333333
  0.          0.          0.16666667  0.5         0.125       0.5         0.1
  0.33333333  0.25        0.25        0.25        1.          1.          0.
  0.6         0.33333333  0.4         0.66666667  0.4         0.4         0.5
  0.42857143  0.4         0.6         0.11111111  0.14285714  0.33333333
  0.28571429  0.33333333  0.2         0.22222222  0.2         0.125
  0.28571429  0.2         0.5         0.57142857  0.16666667  0.2         0.125
  0.33333333  0.33333333  0.2         1.          0.125       0.33333333
  0.25        0.18181818  0.25        0.2         0.375       0.6
  0.16666667  0.125       0.5         0.6         0.1         0.33333333
  0.5         0.33333333  0.28571429  0.33333333  0.14285714  0.6
  0.11111111  0.5         0.5         0.6    

In [20]:
arr=np.zeros((3,3,3))
np.mean(arr)

0.0

In [23]:
[i for i in np.array((20,200))/10]

[2, 20]

In [25]:
total = 0
small_count = 0
for pic in gt:
    for i in xrange(len(pic)):
        total+=1
        bbox = pic[i]
        w = bbox[2]-bbox[0]
        h = bbox[3]-bbox[1]
        thres = 0.5
        alt_thres = float(h*w)/((h+10)*(w+10))
        
        if alt_thres<thres:
            small_count += 1
            alt_thress.append(alt_thres)
#         print(min(thres, alt_thres), h*w)
print('total=', total)
print('small count=', small_count)
print('fraction=', float(small_count)/total)
print('avg alt_thres=', np.average(alt_thress))

('total=', 219)
('small count=', 105)
('fraction=', 0.4794520547945205)
('avg alt_thres=', 0.44045709686314422)


In [24]:
total = 0
small_count = 0
for pic in train_gt:
    for i in xrange(len(pic)):
        total+=1
        bbox = pic[i]
        w = bbox[2]-bbox[0]
        h = bbox[3]-bbox[1]
        thres = 0.5
        alt_thres = float(h*w)/((h+10)*(w+10))
        
        if alt_thres<thres:
            small_count += 1
            alt_thress.append(alt_thres)
#         print(min(thres, alt_thres), h*w)
print('total=', total)
print('small count=', small_count)
print('fraction=', float(small_count)/total)
print('avg alt_thres=', np.average(alt_thress))

('total=', 1320)
('small count=', 655)
('fraction=', 0.4962121212121212)
('avg alt_thres=', 0.4400120239468574)


In [23]:
total = 0
small_count = 0
alt_thress = []
for pic in test_gt:
    for i in xrange(len(pic)):
        total+=1
        bbox = pic[i]
        w = bbox[2]-bbox[0]
        h = bbox[3]-bbox[1]
        thres = 0.5
        alt_thres = float(h*w)/((h+10)*(w+10))
        
        if alt_thres<thres:
            small_count += 1
            alt_thress.append(alt_thres)
#         print(min(thres, alt_thres), h*w)
print('total=', total)
print('small count=', small_count)
print('fraction=', float(small_count)/total)
print('avg alt_thres=', np.average(alt_thress))

('total=', 256)
('small count=', 126)
('fraction=', 0.4921875)
('avg alt_thres=', 0.44031356988801656)


In [30]:
print([d.shape[0] for d in dt])

[8, 11, 13, 19, 1, 2, 6, 2, 5, 6, 6, 10, 4, 3, 2, 1, 4, 5, 3, 7, 6, 2, 7, 4, 7, 3, 3, 7, 3, 0, 6, 6, 13, 2, 3, 3, 2, 7, 9, 3, 3, 7, 4, 4, 1, 2, 5, 2, 6, 2, 2, 3, 3, 5, 2, 2, 3, 3, 4, 6, 1, 7, 4, 1, 5, 1, 2, 7, 7, 5, 14, 8, 5, 12, 14, 4, 8, 12, 9, 2, 6, 16, 14, 4, 5, 15, 3, 9, 6, 10, 6, 7, 10, 8, 4, 3, 5, 7, 3, 4, 9, 8, 2, 3, 4, 8, 6, 3, 3, 8, 5, 6, 6, 6, 5, 3, 9, 7, 7, 9, 12, 7, 18, 2, 7, 6, 5, 3, 7, 5, 7, 5, 5, 4, 6, 6, 4]
